In [ ]:
!pip install torchtext==0.6.0

     |████████████████████████████████| 71kB 10.4MB/s 
     |████████████████████████████████| 1.1MB 23.2MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

In [ ]:
from torchtext.data.metrics import bleu_score

In [ ]:
SEED = 1

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
!python -m spacy download en
!python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 14.9MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907056 sha256=72f5270fe51ef0ac256433bf4f8bf8ea6be5091db4dd833de5c35d8fb644712c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ysok0naj/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [ ]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            include_lengths = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 917kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 274kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 263kB/s]


In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x : len(x.src),
     device = device)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, levels, k, attention):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.cnn_level = nn.Conv1d(enc_hid_dim*2,enc_hid_dim*2,kernel_size=k,stride=k)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.output_level = nn.Linear(enc_hid_dim * 2, enc_hid_dim)

        self.dropout = nn.Dropout(dropout)

        self.dropout2 = nn.Dropout(dropout*1.25)
        
        self.levels = levels

        self.k = k

        self.attention = attention

    def forward(self, src, src_len):
        
        #src = [src len, batch size], src len = max_len
        #src_len = [batch size]
        
        embedded = self.dropout(self.embedding(src))
        batch_size = src.shape[1]
        #embedded = [src len, batch size, emb dim]

        src_len=src_len.cpu()       
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len)
                
        packed_outputs, hidden = self.rnn(packed_embedded)
                              
        #packed_outputs is a packed sequence containing all hidden states
        #hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs)
        enc_hid_dim=list(outputs.shape)[2] 
        #outputs is now a non-packed sequence, all hidden states obtained
        #  when the input is a pad token are all zeros
            
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]


        mask = torch.zeros(list(outputs.shape)[1],list(outputs.shape)[0],device=device)
        #mask = [batch size, src len]
        for q in range(list(outputs.shape)[1]):
          mask[q,:src_len[q]]=1

        length_of_seq = list(outputs.shape)[0]

        if (list(outputs.shape)[0]%self.k !=0):
            
            temp_pad=(outputs.shape[0]//k +1)*k
            padded_outputs=torch.zeros((temp_pad,list(outputs.shape)[1],list(outputs.shape)[2]),device=device)
            padded_outputs[:list(outputs.shape)[0],:,:]=outputs
            mask_temp=torch.zeros((list(outputs.shape)[1], temp_pad),device=device)
            mask_temp[:, :list(mask.shape)[1]]=mask
        else:
            padded_outputs = outputs
            mask_temp = mask
        
        final_outputs=[padded_outputs]
        masks=[mask_temp]

        if (self.levels>1):
            for i in range(1,self.levels):
                
                temp_outputs=self.dropout2(torch.nn.functional.relu(self.cnn_level(final_outputs[-1].permute(1,2,0))))
                # [batch_size, hidden_dim, T/k]
                #print('shape',temp_outputs.shape)
                temp_outputs=temp_outputs.permute(2,0,1)
                length_of_seq = list(temp_outputs.shape)[0]
                #outputs = [src len, batch size, enc hid dim * 2]

                
                
                padded_outputs=final_outputs[-1].permute(1,0,2)
                #padded_outputs=padded_outputs.reshape(batch_size,self.k,-1,enc_hid_dim*2)
                #padded_outputs=padded_outputs.permute(0,2,1,3)
                padded_outputs=padded_outputs.reshape(-1,self.k,enc_hid_dim)
                #[batch size*T/k, k, enc hid dim * 2]
                #print('padded_outputs.shape:',padded_outputs.shape)

                #mask_temp=mask_temp.reshape(batch_size,self.k,-1)
                #mask_temp=mask_temp.permute(0,2,1)
                mask_temp=masks[-1].reshape(-1,self.k)
                #[batch size*T/k, k]

                temp_padded_outputs=temp_outputs
                temp_padded_outputs=temp_padded_outputs.permute(1,0,2)
                #temp_padded_outputs=temp_padded_outputs.unsqueeze(3)
                temp_padded_outputs=temp_padded_outputs.reshape(-1,enc_hid_dim)
                #[batch size*T/k, enc hid dim * 2]

                #make mask
                
                a=self.attention(temp_padded_outputs,padded_outputs.permute(1,0,2),mask_temp)
                a = a.unsqueeze(1)
                weighted = torch.bmm(a, padded_outputs)
                #weighted = [batch size*T/k, 1, enc hid dim * 2]

                weighted=weighted.squeeze(1)
                weighted_pad=weighted.view(batch_size,-1,enc_hid_dim)

                level_outputs = weighted_pad.permute(1,0,2)
                if (list(level_outputs.shape)[0]%self.k ==0):
                    padded_outputs=level_outputs
                else:
                    temp_pad=(list(level_outputs.shape)[0]//k +1)*k
                    padded_outputs=torch.zeros((temp_pad,list(level_outputs.shape)[1],list(level_outputs.shape)[2]),device=device)
                    padded_outputs[:list(level_outputs.shape)[0],:,:]=level_outputs

                #update src_len, masks
                #print(type(src_len))
                b = (src_len // self.k)
                b[src_len % self.k != 0] += 1
                src_len=b
                #print(type(src_len))
                outputs=padded_outputs
                mask = torch.zeros(list(outputs.shape)[1],list(outputs.shape)[0],device=device)  #mask = [batch size, src len]
                for q in range(list(outputs.shape)[1]):
                    mask[q,:src_len[q]]=1
                masks.append(mask)
                final_outputs.append(outputs)
                #print("encoder_outputs:",outputs.shape)
        
        return final_outputs, hidden, masks

    

In [ ]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        #print('self.attn',(enc_hid_dim * 2) + dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
  
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        #print(torch.cat((hidden, encoder_outputs), dim = 2).shape)
        #print('hidden.shape,encoder_outputs.shape:',hidden.shape,encoder_outputs.shape)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return F.softmax(attention, dim = 1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention, k):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)

        self.k=k
        
    def forward(self, input, hidden, encoder_outputs, masks):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = list of this size[src len, batch size, enc hid dim * 2]
        #mask = list of this size[batch size, src len]
        
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        batch_size = list(encoder_outputs[0].shape)[1]
        enc_hid_dim = list(encoder_outputs[0].shape)[2]//2
        prob_remaining=torch.ones(batch_size).to(device)
        #top1=0
        weighted=torch.zeros((batch_size, 1, enc_hid_dim * 2)).to(device)

        batch_number=torch.arange(batch_size,device=device)

        for i in range(len(encoder_outputs)-1,-1,-1):
            
            
            if (i==(len(encoder_outputs)-1)):

                current_outputs = encoder_outputs[i]

                current_mask = masks[i]
            else:
                encoder_out_temp=encoder_outputs[i]
                mask_temp = masks[i]
                '''if (list(encoder_outputs[i].shape)[0]%self.k ==0):
                    encoder_out_temp=encoder_outputs[i]
                    mask_temp=masks[i]
                    
                else:
                    temp_pad=(list(encoder_outputs[i].shape)[0]//k +1)*k
                    encoder_out_temp=torch.zeros((temp_pad,list(encoder_outputs[i].shape)[1],list(encoder_outputs[i].shape)[2]))

                    encoder_out_temp[:list(encoder_outputs[i].shape)[0],:,:]=encoder_outputs[i]
                    mask_temp=torch.zeros((list(encoder_outputs[i].shape)[1], temp_pad))
                    mask_temp[:, :list(masks[i].shape)[1]]=masks[i]'''

                current_outputs=torch.zeros((self.k,list(encoder_outputs[i].shape)[1],list(encoder_outputs[i].shape)[2]), device=device)
                current_mask=torch.zeros((list(encoder_outputs[i].shape)[1],self.k),device=device)
                
                #for z in range(list(top1.shape)[0]):
                for z in range(self.k):
                    #print("encoder_out_temp.shape,current_outputs.shape",encoder_out_temp.shape,current_outputs.shape)
                    #current_outputs[:,z,:] = encoder_out_temp[top1[z]:top1[z]+k,z,:]
            
                    #current_mask[z,:] = mask_temp[z,top1[z]:top1[z]+k]
                   
                    current_outputs[z,:,:] = encoder_out_temp[top1+z,batch_number,:]
            
                    current_mask[:,z] = mask_temp[batch_number,top1+z]
                
                
                #current_outputs = encoder_outputs[i][top1:min(top1+k,list(encoder_outpus[i].shape)[0],:,:]
            
                #current_mask =masks[i][top1:min(top1+k,list(encoder_outpus[i].shape)[0],:,:]
            
            #current_outputs=current_outputs.to(device)
            #current_mask=current_mask.to(device)
            a = self.attention(hidden, current_outputs, current_mask)
                    
            #a = [batch size, src len]
            
            
            top1 = a.argmax(1)
            prob_remaining_prev=prob_remaining
            #print("tuple:",a.max(1),prob_remaining_prev)
            prob_remaining = a.max(1)[0]*prob_remaining_prev #element-wise multiplication
            # [batch_size]

            a = a.unsqueeze(1)
            #a = [batch size, 1, src len]
            
            #current_outputs[top1,:,:]=0 #to not attend to max weights
            current_outputs=current_outputs.clone()
            #start_t = time.time()
            if i!=0:

                current_outputs[top1,batch_number,:]=0
            
            #end_t = time.time()
            #epoch_mins, epoch_secs = epoch_time(start_t, end_t)
            #print('de_timestamp, loop time:',epoch_mins, epoch_secs)
            current_outputs = current_outputs.permute(1, 0, 2)
            
            #encoder_outputs = [batch size, src len, enc hid dim * 2]
            #print("tuple:",a.shape,current_outputs.shape,prob_remaining_prev)

            weighted_temp = torch.bmm(a, current_outputs)*(prob_remaining_prev.view(-1,1,1)) #elementwise multplication+broadcasting
            #weighted = [batch size, 1, enc hid dim * 2]    [batch_size]

            weighted=weighted+weighted_temp
            
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
            
        return prediction, hidden.squeeze(0), a.squeeze(1)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layers
        start_t=time.time()

        encoder_outputs, hidden, masks = self.encoder(src, src_len)

        end_t=time.time()
        epoch_mins, epoch_secs = epoch_time(start_t, end_t)
        #print('en_time:',epoch_mins, epoch_secs)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        mask = self.create_mask(src)

        #mask = [batch size, src len]
        start_t=time.time()       
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs, masks)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted tokenend
            input = trg[t] if teacher_force else top1
        end_t=time.time()
        epoch_mins, epoch_secs = epoch_time(start_t, end_t)
        #print('de_time:',epoch_mins, epoch_secs)   
        return outputs

In [ ]:
#change here
k=3
levels=2
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]

dec_attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc_attn = Attention(ENC_HID_DIM, ENC_HID_DIM * 2)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT,levels,k,enc_attn)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, dec_attn,k)

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (cnn_level): Conv1d(1024, 1024, kernel_size=(3,), stride=(3,))
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (output_level): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dropout2): Dropout(p=0.625, inplace=False)
    (attention): Attention(
      (attn): Linear(in_features=2048, out_features=1024, bias=True)
      (v): Linear(in_features=1024, out_features=1, bias=False)
    )
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 26,289,669 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)


In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    with torch.autograd.set_detect_anomaly(True):
        print('no. of batches:', len(iterator))
        for i, batch in enumerate(iterator):
            src, src_len = batch.src
            #print('batch_no:',i+1,'max_len:',max(src_len),'min_len',min(src_len))
            trg = batch.trg
            optimizer.zero_grad()
            
            output = model(src, src_len, trg)
            
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]
            
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]
            
            loss = criterion(output, trg)
            
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            
            optimizer.step()
            
            epoch_loss += loss.item()
            
        return epoch_loss / len(iterator)
        

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_len = batch.src
            print('batch_no:',i+1,'max_len:',max(src_len),'min_len',min(src_len))
            trg = batch.trg

            output = model(src, src_len, trg, 0) #turn off teacher forcing
            
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = (elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

no. of batches: 227
batch_no: 1 max_len: tensor(10, device='cuda:0') min_len tensor(5, device='cuda:0')
batch_no: 2 max_len: tensor(12, device='cuda:0') min_len tensor(10, device='cuda:0')
batch_no: 3 max_len: tensor(13, device='cuda:0') min_len tensor(12, device='cuda:0')
batch_no: 4 max_len: tensor(14, device='cuda:0') min_len tensor(13, device='cuda:0')
batch_no: 5 max_len: tensor(15, device='cuda:0') min_len tensor(14, device='cuda:0')
batch_no: 6 max_len: tensor(17, device='cuda:0') min_len tensor(15, device='cuda:0')
batch_no: 7 max_len: tensor(20, device='cuda:0') min_len tensor(17, device='cuda:0')
batch_no: 8 max_len: tensor(35, device='cuda:0') min_len tensor(20, device='cuda:0')
Epoch: 01 | Time: 3m 19.3422634601593s
	Train Loss: 5.102 | Train PPL: 164.392
	 Val. Loss: 4.789 |  Val. PPL: 120.160
no. of batches: 227
batch_no: 1 max_len: tensor(10, device='cuda:0') min_len tensor(5, device='cuda:0')
batch_no: 2 max_len: tensor(12, device='cuda:0') min_len tensor(10, device='cu

In [ ]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

batch_no: 1 max_len: tensor(10, device='cuda:0') min_len tensor(7, device='cuda:0')
batch_no: 2 max_len: tensor(11, device='cuda:0') min_len tensor(10, device='cuda:0')
batch_no: 3 max_len: tensor(12, device='cuda:0') min_len tensor(11, device='cuda:0')
batch_no: 4 max_len: tensor(13, device='cuda:0') min_len tensor(12, device='cuda:0')
batch_no: 5 max_len: tensor(15, device='cuda:0') min_len tensor(14, device='cuda:0')
batch_no: 6 max_len: tensor(16, device='cuda:0') min_len tensor(15, device='cuda:0')
batch_no: 7 max_len: tensor(19, device='cuda:0') min_len tensor(16, device='cuda:0')
batch_no: 8 max_len: tensor(33, device='cuda:0') min_len tensor(19, device='cuda:0')
| Test Loss: 3.372 | Test PPL:  29.132 |


In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)]).to(device)
    
    with torch.no_grad():
        encoder_outputs, hidden,masks = model.encoder(src_tensor, src_len)

    mask = model.create_mask(src_tensor)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    #attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, masks)

        #attentions[i] = attention
            
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:]#, attentions[:len(trg_tokens)-1]

In [ ]:
def display_attention(sentence, translation, attention):
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    
    attention = attention.squeeze(1).cpu().detach().numpy()
    
    cax = ax.matshow(attention, cmap='bone')
   
    ax.tick_params(labelsize=15)
    ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                       rotation=45)
    ax.set_yticklabels(['']+translation)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()

In [ ]:
example_idx = 12

src = vars(train_data.examples[example_idx])['src']
trg = vars(train_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['ein', 'schwarzer', 'hund', 'und', 'ein', 'gefleckter', 'hund', 'kämpfen', '.']
trg = ['a', 'black', 'dog', 'and', 'a', 'spotted', 'dog', 'are', 'fighting']


In [ ]:
#translation, attention = translate_sentence(src, SRC, TRG, model, device)
translation = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

predicted trg = ['a', 'black', 'dog', 'and', 'a', 'spotted', 'dog', 'fighting', '.', '<eos>']


In [ ]:
example_idx = 14

src = vars(valid_data.examples[example_idx])['src']
trg = vars(valid_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['eine', 'frau', 'spielt', 'ein', 'lied', 'auf', 'ihrer', 'geige', '.']
trg = ['a', 'female', 'playing', 'a', 'song', 'on', 'her', 'violin', '.']


In [ ]:
#translation, attention = translate_sentence(src, SRC, TRG, model, device)
translation = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

#display_attention(src, translation, attention)

predicted trg = ['a', 'woman', 'is', 'playing', 'a', 'violin', 'on', 'violin', '.', '<eos>']


In [ ]:
example_idx = 18

src = vars(test_data.examples[example_idx])['src']
trg = vars(test_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

src = ['die', 'person', 'im', 'gestreiften', 'shirt', 'klettert', 'auf', 'einen', 'berg', '.']
trg = ['the', 'person', 'in', 'the', 'striped', 'shirt', 'is', 'mountain', 'climbing', '.']


In [ ]:
#translation, attention = translate_sentence(src, SRC, TRG, model, device)
translation = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

#display_attention(src, translation, attention)

predicted trg = ['the', 'person', 'in', 'a', 'striped', 'shirt', 'is', 'climbing', 'a', 'rock', '.', '<eos>']


In [ ]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        #pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
        pred_trg = translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return bleu_score(pred_trgs, trgs)

In [ ]:
bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

BLEU score = 22.70
